In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import csv
import urllib.request
import seaborn as sns
import pandas as pd
import json
import re
import os
import glob
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
# import zipfile

# zip_path = '/content/drive/MyDrive/Index/europarl.zip'
# extract_path = '/content/drive/MyDrive/Index/europarl'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_path)

# **Exercice 1 : Construire un index**

In [ ]:
def acces_fichier(chemin):
    fichier = open(chemin, encoding="utf8")
    contenu = fichier.read()
    fichier.close()
    return contenu

In [ ]:
def decoup (chaine):
    return chaine.split(' ')

In [ ]:
import glob
import os
import chardet

def acces_fichier(chemin):
    with open(chemin, 'rb') as fichier:
        raw_data = fichier.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    with open(chemin, encoding=encoding, errors='ignore') as fichier:
        contenu = fichier.read()
    return contenu

def decoup(chaine):
    return chaine.split()

index = {}
sous_dossier = "/content/drive/MyDrive/Index/europarl/europarl/fr"

# Récupérer tous les fichiers dans le sous-dossier et ses sous-dossiers
chemins = [os.path.join(root, file)
           for root, dirs, files in os.walk(sous_dossier)
           for file in files]

for chemin in chemins:
    if os.path.isfile(chemin):
        chaine = acces_fichier(chemin)
        mots = decoup(chaine)
        for mot in mots:
            if mot not in index:
                index[mot] = set()
            index[mot].add(chemin)

# Afficher la taille de l'index
print(len(index))

# Afficher les 50 premières clés de l'index
print(list(index.keys())[:50])


In [ ]:
print(index["indique"])

In [ ]:
print(index["européenne"])

In [ ]:
print(index["toto"])

{'/content/drive/MyDrive/Index/europarl/europarl/fr/ep-02-10-22-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-00-01-18-fr.txt'}


In [ ]:
index = {mot:list( liste_fichiers )
for mot , liste_fichiers in index.items ()}
w = open("index.json", "w")
w.write(json.dumps(index ))
w.close ()

In [ ]:
comEur="Commission Europeenne"
nv=decoup(comEur)

print(nv)

['Commission', 'Europeenne']


In [ ]:
print(index[nv[0]])

# **Exercice 3 : Pr´esenter les contextes**

In [ ]:
import re

def afficher_contextes(chaine, terme, taille_contexte=30):
    match = re.search(terme, chaine)
    contexts = []
    while match is not None:
        # Les bornes gauche et droite autour du mot
        gauche = max(match.start() - taille_contexte, 0)
        droite = match.end() + taille_contexte
        contexts.append(chaine[gauche:droite])
        chaine = chaine[match.end():]
        match = re.search(terme, chaine)
    return contexts


In [ ]:
terme = "commission "

for chemin in chemins:
    if os.path.isfile(chemin):
        chaine = acces_fichier(chemin)
        contexts = afficher_contextes(chaine, terme)
        for c in contexts:
            print(contexts)


# **Exercice 4 : Du texte au token**

In [ ]:
import nltk
from nltk.corpus import stopwords

# Assurez-vous d'avoir téléchargé les ressources nécessaires
nltk.download('punkt')
nltk.download('stopwords')

# Exemple de contenu texte
texte = """
Reprise de la session
Je déclare reprise la session du Parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vœux en espérant que vous avez passé de bonnes vacances.
Comme vous avez pu le constater, le grand "bogue de l'an 2000" ne s'est pas produit.
"""

# Tokenisation du texte en mots
tokens = nltk.word_tokenize(texte, language='french')

# Suppression des mots vides
tokens = [w for w in tokens if w.lower() not in stopwords.words('french')]

print(tokens)


# **Exercice 5 : Construire l’index et l’index invers´e**

In [ ]:
import os
import nltk
from nltk.corpus import stopwords
import chardet
import time

# Assurez-vous d'avoir téléchargé les ressources nécessaires
nltk.download('punkt')
nltk.download('stopwords')

def acces_fichier(chemin):
    with open(chemin, 'rb') as fichier:
        raw_data = fichier.read()
        result = chardet.detect(raw_data)
        encoding = result['encoding']

    with open(chemin, encoding=encoding, errors='ignore') as fichier:
        contenu = fichier.read()
    return contenu

def creer_index(sous_dossier):
    index = {}
    chemins = [os.path.join(root, file)
               for root, dirs, files in os.walk(sous_dossier)
               for file in files]

    total_files = len(chemins)
    print(f"Nombre total de fichiers à traiter: {total_files}")

    for i, chemin in enumerate(chemins):
        if os.path.isfile(chemin):
            texte = acces_fichier(chemin)
            tokens = nltk.word_tokenize(texte, language='french')
            tokens = [w.lower() for w in tokens if w.lower() not in stopwords.words('french')]
            for mot in tokens:
                if mot not in index:
                    index[mot] = set()
                index[mot].add(chemin)

        if (i + 1) % 100 == 0 or (i + 1) == total_files:
            print(f"Fichiers traités: {i + 1}/{total_files}")

    return index

def creer_index_inverse(index):
    index_inverse = {}
    for mot, chemins in index.items():
        for chemin in chemins:
            if chemin not in index_inverse:
                index_inverse[chemin] = set()
            index_inverse[chemin].add(mot)
    return index_inverse

def requeter_documents(requete, index):
    if requete in index:
        return list(index[requete])
    else:
        return []

# Modifier le chemin selon votre configuration
sous_dossier = "/content/drive/MyDrive/Index/europarl/europarl/fr"

# Créer l'index et l'index inverse
start_time = time.time()
index = creer_index(sous_dossier)
index_inverse = creer_index_inverse(index)
end_time = time.time()

print(f"Temps total d'exécution: {end_time - start_time} secondes")
print("Nombre de termes différents :", len(index.keys()))
print("Nombre de documents :", len(index_inverse.keys()))

# Exemple d'utilisation de la fonction requeter_documents
requete1 = "commission"
docs_trouves = requeter_documents(requete1, index)
print(f"Nombre de documents trouvés pour la requête '{requete1}' :", len(docs_trouves))

requete2 = "européenne"
docs_trouves = requeter_documents(requete2, index)
print(f"Nombre de documents trouvés pour la requête '{requete2}' :", len(docs_trouves))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Nombre total de fichiers à traiter: 364
Fichiers traités: 100/364
Fichiers traités: 200/364
Fichiers traités: 300/364
Fichiers traités: 364/364
Temps total d'exécution: 5008.920107603073 secondes
Nombre de termes différents : 112387
Nombre de documents : 364
Nombre de documents trouvés pour la requête 'commission' : 362
Nombre de documents trouvés pour la requête 'européenne' : 363


In [ ]:
requete = "commission"
docs_trouves = requeter_documents (requete , index)
print (" Nombre de documents trouvés :", len( docs_trouves ))
requete = "européenne"
docs_trouves = requeter_documents (requete , index)
print (" Nombre de documents trouvés :", len( docs_trouves ))

 Nombre de documents trouvés : 362
 Nombre de documents trouvés : 363


In [ ]:

allDocs ={ "docA":{"Le":0.25 ,"petit":0.25 ,"chat":0.25 ,"dort" :0.25},
"docB": {"Le":0.3333 , "chat":0.3333 , "dort" :0.3333},
"docC": {"Jean":0.5 , "dort" :0.5}}


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
df=pd.DataFrame(allDocs)

In [ ]:
df

,docA,docB,docC
Le,0.25,0.3333,NaN
petit,0.25,NaN,NaN
chat,0.25,0.3333,NaN
dort,0.25,0.3333,0.5
Jean,NaN,NaN,0.5


In [ ]:
docA='Le petit chat dort'
docB='Le chat dort'
docC='Jean dort'

In [ ]:
phraseA=docA.split(' ')
phraseB=docB.split(' ')
phraseC=docC.split(' ')

In [ ]:
uniqueWords = set(docA).union(set(docB))
print(uniqueWords)

{' ', 'e', 't', 'r', 'i', 'd', 'h', 'p', 'o', 'a', 'c', 'L'}


In [ ]:
import math
print(f'The value of pi is approximately {math.pi:.3f}.')

The value of pi is approximately 3.142.


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
from nltk.corpus import stopwords
stopwords.words('french')

['au',
 'aux',
 'avec',
 'ce',
 'ces',
 'dans',
 'de',
 'des',
 'du',
 'elle',
 'en',
 'et',
 'eux',
 'il',
 'ils',
 'je',
 'la',
 'le',
 'les',
 'leur',
 'lui',
 'ma',
 'mais',
 'me',
 'même',
 'mes',
 'moi',
 'mon',
 'ne',
 'nos',
 'notre',
 'nous',
 'on',
 'ou',
 'par',
 'pas',
 'pour',
 'qu',
 'que',
 'qui',
 'sa',
 'se',
 'ses',
 'son',
 'sur',
 'ta',
 'te',
 'tes',
 'toi',
 'ton',
 'tu',
 'un',
 'une',
 'vos',
 'votre',
 'vous',
 'c',
 'd',
 'j',
 'l',
 'à',
 'm',
 'n',
 's',
 't',
 'y',
 'été',
 'étée',
 'étées',
 'étés',
 'étant',
 'étante',
 'étants',
 'étantes',
 'suis',
 'es',
 'est',
 'sommes',
 'êtes',
 'sont',
 'serai',
 'seras',
 'sera',
 'serons',
 'serez',
 'seront',
 'serais',
 'serait',
 'serions',
 'seriez',
 'seraient',
 'étais',
 'était',
 'étions',
 'étiez',
 'étaient',
 'fus',
 'fut',
 'fûmes',
 'fûtes',
 'furent',
 'sois',
 'soit',
 'soyons',
 'soyez',
 'soient',
 'fusse',
 'fusses',
 'fût',
 'fussions',
 'fussiez',
 'fussent',
 'ayant',
 'ayante',
 'ayantes',


# **Exercice 6 : Calculer les TF et IDF**

In [ ]:
import math

# Exemple de données TF pour chaque document
allDocs = {
    "docA": {"Le": 0.25, "petit": 0.25, "chat": 0.25, "dort": 0.25},
    "docB": {"Le": 0.3333, "chat": 0.3333, "dort": 0.3333},
    "docC": {"Jean": 0.5, "dort": 0.5}
}

# Exemple de données IDF pour chaque terme
mot2idf = {
    "Le": 0.17609,
    "petit": 0.47712,
    "Jean": 0.47712,
    "chat": 0.17609,
    "dort": 0.0
}

def calculer_tf_idf(tf, idf):
    tf_idf = {}
    for terme, valeur_tf in tf.items():
        if terme in idf:
            tf_idf[terme] = valeur_tf * idf[terme]
        else:
            tf_idf[terme] = 0.0  # En cas où IDF n'est pas défini pour un terme (ce qui ne devrait pas arriver ici)
    return tf_idf

# Calculer TF-IDF pour chaque document
for doc, tf in allDocs.items():
    tf_idf = calculer_tf_idf(tf, mot2idf)
    print(f"TF-IDF pour {doc}: {tf_idf}")


TF-IDF pour docA: {'Le': 0.0440225, 'petit': 0.11928, 'chat': 0.0440225, 'dort': 0.0}
TF-IDF pour docB: {'Le': 0.058690796999999996, 'chat': 0.058690796999999996, 'dort': 0.0}
TF-IDF pour docC: {'Jean': 0.23856, 'dort': 0.0}


In [ ]:
import math

# Exemple de vecteurs TF-IDF pour deux documents
docA = {"Le": 0.25, "petit": 0.25, "chat": 0.25, "dort": 0.25}
docB = {"Le": 0.3333, "chat": 0.3333, "dort": 0.3333}

def calculer_produit_scalaire(doc1, doc2):
    produit_scalaire = 0.0
    for terme in doc1:
        if terme in doc2:
            produit_scalaire += doc1[terme] * doc2[terme]
    return produit_scalaire

def calculer_norme(vecteur):
    norme = math.sqrt(sum(value ** 2 for value in vecteur.values()))
    return norme if norme != 0 else 1  # Pour éviter une division par zéro si le vecteur est vide

def calculer_similarite_cosinus(doc1, doc2):
    produit_scalaire = calculer_produit_scalaire(doc1, doc2)
    norme_doc1 = calculer_norme(doc1)
    norme_doc2 = calculer_norme(doc2)

    if norme_doc1 == 0 or norme_doc2 == 0:
        return 0.0

    similarite_cosinus = produit_scalaire / (norme_doc1 * norme_doc2)
    return similarite_cosinus

# Calculer la similarité cosinus entre docA et docB
sim_cosinus = calculer_similarite_cosinus(docA, docB)
print(f"Taux de similarité cosinus entre docA et docB : {sim_cosinus}")


Taux de similarité cosinus entre docA et docB : 0.8660254037844387


In [ ]:
print("Clés dans documents_trouves :", documents_trouves.keys())
print("Clés dans index_inverse :", index_inverse.keys())

Clés dans documents_trouves : dict_keys(['docA', 'docB', 'docC'])
Clés dans index_inverse : dict_keys(['/content/drive/MyDrive/Index/europarl/europarl/fr/ep-01-06-14-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-02-07-01-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-02-04-09-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-03-10-23-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-05-09-06-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-02-09-03-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-03-06-03-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-00-07-04-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-00-01-20-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-04-03-30-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-05-07-06-fr.txt', '/content/drive/MyDrive/Index/europarl/europarl/fr/ep-02-07-03-fr.txt', '/content/drive/MyDrive/Index/eur

In [ ]:
index_inverse = {
    "docA": ["Le", "petit", "chat", "dort"],
    "docB": ["Le", "chat", "dort"],
    "docC": ["Jean", "dort"]
}



# **Exercice 7 : Calcul du taux de similarit´e cosinus**

In [ ]:
import math

# Exemple de vecteur TF-IDF pour une requête
requete = {"Le": 0.2, "petit": 0.1, "chat": 0.3, "dort": 0.4}

# Exemple de documents trouvés avec leurs vecteurs TF-IDF
documents_trouves = {
    "docA": {"Le": 0.25, "petit": 0.25, "chat": 0.25, "dort": 0.25},
    "docB": {"Le": 0.3333, "chat": 0.3333, "dort": 0.3333},
    "docC": {"Jean": 0.5, "dort": 0.5}
}

def calculer_similarites_cosinus(requete, documents):
    sim_cosinus = {}
    for doc, vecteur in documents.items():
        sim_cosinus[doc] = calculer_similarite_cosinus(requete, vecteur)
    return sim_cosinus

def calculer_ponderation_cosinus(requete, index_inverse, documents_trouves):
    sim_cosinus = calculer_similarites_cosinus(requete, documents_trouves)
    docs_pond = {}
    for doc, sim in sim_cosinus.items():
        docs_pond[doc] = sim * len(index_inverse[doc])  # Pondération par le nombre de termes dans le document
    return docs_pond

# Fonction pour trier les documents par pertinence décroissante
def trier_documents_par_pertinence(docs_pond):
    docs_pond_liste = [[sim, chemin] for chemin, sim in docs_pond.items()]
    docs_pond_liste = sorted(docs_pond_liste, reverse=True)
    return docs_pond_liste

# Calculer la pondération cosinus pour la requête et les documents trouvés
docs_pond = calculer_ponderation_cosinus(requete, index_inverse, documents_trouves)

# Trier les documents par pertinence décroissante
docs_pond_liste = trier_documents_par_pertinence(docs_pond)

# Afficher les résultats triés
for sim, chemin in docs_pond_liste:
    print(f"Similitude cosinus : {sim}, Document : {chemin}")


Similitude cosinus : 3.651483716701107, Document : docA
Similitude cosinus : 2.846049894151541, Document : docB
Similitude cosinus : 1.0327955589886444, Document : docC


# **Exercice 8 : Classement par pertinence (ranking)**

In [ ]:
import math

# Exemple de vecteur TF-IDF pour une requête
requete = {"Le": 0.2, "petit": 0.1, "chat": 0.3, "dort": 0.4}

# Exemple de documents trouvés avec leurs vecteurs TF-IDF
documents_trouves = {
    "docA": {"Le": 0.25, "petit": 0.25, "chat": 0.25, "dort": 0.25},
    "docB": {"Le": 0.3333, "chat": 0.3333, "dort": 0.3333},
    "docC": {"Jean": 0.5, "dort": 0.5}
}

# Calculer la similarité cosinus entre deux vecteurs TF-IDF
def calculer_similarite_cosinus(vecteur1, vecteur2):
    produit_scalaire = sum(vecteur1[term] * vecteur2.get(term, 0) for term in vecteur1)
    norme_vecteur1 = math.sqrt(sum(vecteur1[term] ** 2 for term in vecteur1))
    norme_vecteur2 = math.sqrt(sum(vecteur2[term] ** 2 for term in vecteur2))
    if norme_vecteur1 != 0 and norme_vecteur2 != 0:
        return produit_scalaire / (norme_vecteur1 * norme_vecteur2)
    else:
        return 0

# Calculer les similarités cosinus entre la requête et les documents trouvés
def calculer_similarites_cosinus(requete, documents):
    sim_cosinus = {}
    for doc, vecteur in documents.items():
        sim_cosinus[doc] = calculer_similarite_cosinus(requete, vecteur)
    return sim_cosinus

# Fonction pour trier les documents par pertinence décroissante
def trier_documents_par_pertinence(requete, documents_trouves):
    sim_cosinus = calculer_similarites_cosinus(requete, documents_trouves)
    docs_pond = [[sim, chemin] for chemin, sim in sim_cosinus.items()]
    docs_pond = sorted(docs_pond, key=lambda x: x[0], reverse=True)
    return docs_pond

# Appeler la fonction pour trier les documents par pertinence
docs_pond_liste = trier_documents_par_pertinence(requete, documents_trouves)

# Afficher les résultats triés
for sim, chemin in docs_pond_liste:
    print(f"{sim} {chemin}")



0.9486832980505137 docB
0.9128709291752768 docA
0.5163977794943222 docC


# **Exercice 9 : Structuration de l’espace de recherche**

In [ ]:
from sklearn.neighbors import KDTree
import numpy as np

# Exemple de données TF-IDF (vecteurs)
tfidf_vectors = np.array([
    [0.25, 0.25, 0.25, 0.25],  # docA
    [0.3333, 0, 0.3333, 0.3333],  # docB
    [0, 0, 0.5, 0.5]  # docC
])

# Construire l'index KD-Tree
kd_tree = KDTree(tfidf_vectors)

# Exemple de requête (vecteur TF-IDF)
requete = np.array([[0.2, 0.1, 0.3, 0.4]])

# Trouver les k plus proches voisins
k_neighbors = kd_tree.query(requete, k=2, return_distance=False)

# Afficher les résultats
for neighbor_index in k_neighbors[0]:
    print(f"Document le plus proche : doc{neighbor_index + 1}")


Document le plus proche : doc2
Document le plus proche : doc1
